## INFORME RETIRO

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import glob
from os import getcwd

%matplotlib inline

#pd.options.display.max_rows = 10
pd.options.display.float_format = '{:,.2f}'.format
plt.rcParams['figure.figsize'] = (7, 5)

In [2]:
lista_balances = glob.glob(r'input\BALANCES\*.mdb')

In [3]:
lista_balances

['input\\BALANCES\\2018-2.mdb',
 'input\\BALANCES\\2018-3.mdb',
 'input\\BALANCES\\2018-4.mdb',
 'input\\BALANCES\\2019-1.mdb',
 'input\\BALANCES\\2019-2.mdb']

In [4]:
# Nombre del controlador.
DRIVER_NAME = "Microsoft Access Driver (*.mdb, *.accdb)"

# Ruta completa del archivo.
DB_PATH = 'input\\BALANCES\\2018-2.mdb'

# Establecer la conexión.
conn = pyodbc.connect("Driver={%s};DBQ=%s;" % (DRIVER_NAME, DB_PATH))
SQL_Query = pd.read_sql_query('''select * from Balance;''', conn)
    
#convierto la query en dataframe
Balance2018_2 = pd.DataFrame(SQL_Query)

conn.close()

In [5]:
# Nombre del controlador.
DRIVER_NAME = "Microsoft Access Driver (*.mdb, *.accdb)"

# Ruta completa del archivo.
DB_PATH = 'input\\BALANCES\\2019-1.mdb'

# Establecer la conexión.
conn = pyodbc.connect("Driver={%s};DBQ=%s;" % (DRIVER_NAME, DB_PATH))
SQL_Query = pd.read_sql_query('''select * from Balance;''', conn)
    
#convierto la query en dataframe
Balance2019_1 = pd.DataFrame(SQL_Query)

conn.close()

In [40]:
# Nombre del controlador.
DRIVER_NAME = "Microsoft Access Driver (*.mdb, *.accdb)"

# Ruta completa del archivo.
DB_PATH = 'input\\BALANCES\\2019-2.mdb'

# Establecer la conexión.
conn = pyodbc.connect("Driver={%s};DBQ=%s;" % (DRIVER_NAME, DB_PATH))
SQL_Query = pd.read_sql_query('''select * from Balance;''', conn)
    
#convierto la query en dataframe
Balance2019_2 = pd.DataFrame(SQL_Query)

conn.close()

In [7]:
Balance= Balance2018_2.append(Balance2019_1)
Balance= Balance.append(Balance2019_2)

In [41]:
Balance.tail()

,cod_cia,razon_social,periodo,cod_subramo,desc_subramo,importe,cod_cuenta,desc_cuenta,nivel,id_padre
220639,0922,SOLVENCIA COMPAÑÍA DE SEGUROS DE RETIRO SOCIED...,2019-2,3.000.99,Secc. Admin. - Secc. Admnin.,0.00,5.01.04.04.04.06.01.00,Stros. Pendientes del Ej. Anterior Total - Se...,7,1615
220640,0922,SOLVENCIA COMPAÑÍA DE SEGUROS DE RETIRO SOCIED...,2019-2,3.000.99,Secc. Admin. - Secc. Admnin.,0.00,5.01.04.04.04.11.01.01,Rs. en Curso,8,1637
220641,0922,SOLVENCIA COMPAÑÍA DE SEGUROS DE RETIRO SOCIED...,2019-2,3.000.99,Secc. Admin. - Secc. Admnin.,0.00,5.01.04.04.04.12.01.01,Rs. en Curso,8,1647
220642,0922,SOLVENCIA COMPAÑÍA DE SEGUROS DE RETIRO SOCIED...,2019-2,3.000.99,Secc. Admin. - Secc. Admnin.,0.00,5.01.04.04.04.12.03.01,Reserva Técnica por Insuficiencia de Primas,8,1655
220643,0922,SOLVENCIA COMPAÑÍA DE SEGUROS DE RETIRO SOCIED...,2019-2,3.000.99,Secc. Admin. - Secc. Admnin.,0.00,5.01.04.04.04.14.01.00,Stros. Ocurridos y no Reportados del Ej. Anterior,7,1675


In [4]:
Balance.dtypes

cod_cia          object
razon_social     object
periodo          object
cod_subramo      object
desc_subramo     object
importe         float64
cod_cuenta       object
desc_cuenta      object
nivel             int64
id_padre         object
dtype: object

In [42]:
Balance['cod_cia'] = Balance['cod_cia'].astype(int)

In [43]:
#Balance=Balance[(Balance['periodo']=='2018-2')|(Balance['periodo']=='2019-1')|(Balance['periodo']=='2019-2')]

In [44]:
den_corta=pd.read_excel(r'input\ART\Dataset_Compañías.xlsx', usecols='A,B')

In [45]:
Balance=Balance.merge(den_corta, how='left', on=['cod_cia'])

In [46]:
Balance.columns

Index(['cod_cia', 'razon_social', 'periodo', 'cod_subramo', 'desc_subramo',
       'importe', 'cod_cuenta', 'desc_cuenta', 'nivel', 'id_padre',
       'denominacion_corta'],
      dtype='object')

In [47]:
Balance= Balance[['cod_cia','denominacion_corta','periodo','cod_subramo','desc_subramo','cod_cuenta','desc_cuenta','importe']]

In [49]:
Balance.head()

,cod_cia,denominacion_corta,periodo,cod_subramo,desc_subramo,cod_cuenta,desc_cuenta,importe
0,2,SMG,2019-2,,,1.00.00.00.00.00.00.00,ACTIVO,"6,045,512,945.00"
1,2,SMG,2019-2,,,1.01.00.00.00.00.00.00,DISPONIBILIDADES,"275,225,043.00"
2,2,SMG,2019-2,,,1.01.01.00.00.00.00.00,Caja,"15,113,633.00"
3,2,SMG,2019-2,,,1.01.01.01.00.00.00.00,Caja,"15,113,633.00"
4,2,SMG,2019-2,,,1.01.01.01.01.00.00.00,Caja,"15,113,633.00"


In [50]:
cias_retiro=[419,420,422,423,425,426,433,436,437,442,443,484,530,534,604,710]
Balance['cias_retiro']=[1 if np.isin(Balance.cod_cia.iloc[i],cias_retiro)==True else 0 for i in range(len(Balance.index))]

Balance=Balance[(Balance.cias_retiro==1)]

In [51]:
#ctas1=['4.01.02.02.02.01.00.00','4.01.02.02.02.02.00.00','4.01.02.02.02.03.00.00','4.01.02.02.02.04.00.00','4.01.02.02.02.05.00.00','4.01.02.02.02.50.00.00','4.01.02.03.03.01.00.00','4.01.02.03.03.02.00.00','4.01.02.03.03.03.00.00','4.01.02.03.03.04.00.00','4.01.02.03.03.50.00.00']

#ctas2= ['2.02.02.02.03.01.00.00','2.02.02.02.03.02.00.00','2.02.02.02.03.03.00.00','2.02.02.02.03.04.00.00','2.02.02.02.03.05.00.00','2.02.02.02.03.06.00.00','2.02.02.02.03.20.00.00','2.02.02.02.03.21.00.00','2.02.02.02.03.22.00.00','2.02.02.02.03.26.00.00','2.02.02.02.03.30.00.00','2.02.02.02.03.31.00.00','4.01.02.02.02.01.00.00','4.01.02.02.02.02.00.00','4.01.02.02.02.03.00.00','4.01.02.02.02.04.00.00']

ctas3= ['1.00.00.00.00.00.00.00','1.01.00.00.00.00.00.00','2.00.00.00.00.00.00.00','2.01.01.00.00.00.00.00','2.01.02.00.00.00.00.00','2.02.00.00.00.00.00.00']


Balance['ctas']=[1 if np.isin(Balance.cod_cuenta.iloc[i],ctas3)==True else 0 for i in range(len(Balance.index))]

Balance=Balance[(Balance['ctas']==1)]

In [52]:
Balance=Balance.drop({'cias_retiro','ctas'}, 1)
Balance=Balance.replace(to_replace=np.nan, value=0)

In [20]:
#RM=pd.read_excel(r'input\RETIRO\Reserva Matematica.xlsx', usecols='A,C')

In [21]:
#Balance=Balance.merge(RM, how='left', on=['cod_cuenta'])

In [53]:
Balance=Balance.groupby(['cod_cia','denominacion_corta', 'cod_cuenta', 'desc_cuenta'], as_index=False)['importe'].sum()

In [54]:
Balance

,cod_cia,denominacion_corta,cod_cuenta,desc_cuenta,importe
0,419,ESTRELLA RETIRO,1.00.00.00.00.00.00.00,ACTIVO,"36,139,504,295.00"
1,419,ESTRELLA RETIRO,1.01.00.00.00.00.00.00,DISPONIBILIDADES,"55,836,820.00"
2,419,ESTRELLA RETIRO,2.00.00.00.00.00.00.00,PASIVO,"35,054,267,339.00"
3,419,ESTRELLA RETIRO,2.01.01.00.00.00.00.00,Con Asegurados,"48,080,381.00"
4,419,ESTRELLA RETIRO,2.01.02.00.00.00.00.00,Con Reaseguradores,0.00
5,419,ESTRELLA RETIRO,2.02.00.00.00.00.00.00,COMPROMISOS TECNICOS,"31,593,337,236.00"
6,420,METLIFE RETIRO,1.00.00.00.00.00.00.00,ACTIVO,"13,722,987,630.00"
7,420,METLIFE RETIRO,1.01.00.00.00.00.00.00,DISPONIBILIDADES,"15,998,460.00"
8,420,METLIFE RETIRO,2.00.00.00.00.00.00.00,PASIVO,"12,084,580,250.00"
9,420,METLIFE RETIRO,2.01.01.00.00.00.00.00,Con Asegurados,"120,359,671.00"


In [24]:
#Balance.to_excel(r'C:\Users\eangenello\Desktop\Retiro_1.xlsx', sheet_name='Retiro', index=False)

In [55]:
Balance2=Balance.copy()

In [56]:
Balance2=Balance2.pivot_table(index=['denominacion_corta'], values=['importe'], columns=['desc_cuenta'], aggfunc=np.sum)

In [57]:
Balance2=Balance2.replace(to_replace=np.nan, value=0)

In [58]:
Balance2

importe                                      \
desc_cuenta                      ACTIVO COMPROMISOS TECNICOS Con Asegurados   
denominacion_corta                                                            
BINARIA RETIRO         2,854,095,551.00     2,478,789,576.00   3,636,623.00   
CREDICOOP RETIRO       3,044,379,178.00     2,180,268,448.00  24,430,854.00   
ESTRELLA RETIRO       36,139,504,295.00    31,593,337,236.00  48,080,381.00   
EXPERTA RETIRO            36,760,399.00                 0.00      17,438.00   
FED. PATRONAL RETIRO     122,979,976.00         7,306,913.00     830,642.00   
GALICIA RETIRO           309,150,339.00       211,741,312.00      15,287.00   
HSBC RETIRO           17,424,658,940.00    13,652,227,565.00 118,380,557.00   
METLIFE  RETIRO       13,722,987,630.00    11,155,699,222.00 120,359,671.00   
NACIÓN RETIRO         12,983,929,055.00    10,463,380,169.00 100,081,978.00   
ORÍGENES RETIRO       26,253,250,706.00    23,684,094,010.00 149,890,738.00   
PROYECCIÓN RETIRO        624,496,271.00       473,246,663.00   1,050,573.00   
SEGUNDA RETIRO         2,841,147,458.00     2,468,162,392.00   5,739,062.00   
SMG RETIRO             4,218,527,706.00     2,996,099,298.00  23,591,578.00   
ZURICH RETIRO            108,290,943.00        23,091,652.00           0.00   

                                                                             
desc_cuenta           Con Reaseguradores DISPONIBILIDADES            PASIVO  
denominacion_corta                                                           
BINARIA RETIRO                    680.00    23,104,012.00  2,607,799,120.00  
CREDICOOP RETIRO                    0.00    39,614,702.00  2,384,521,777.00  
ESTRELLA RETIRO                     0.00    55,836,820.00 35,054,267,339.00  
EXPERTA RETIRO                      0.00       755,429.00      1,399,758.00  
FED. PATRONAL RETIRO                0.00     2,255,888.00     34,246,601.00  
GALICIA RETIRO                      0.00       608,334.00    238,168,931.00  
HSBC RETIRO                         0.00    96,547,519.00 14,350,667,466.00  
METLIFE  RETIRO                     0.00    15,998,460.00 12,084,580,250.00  
NACIÓN RETIRO                       0.00    12,052,486.00 11,284,148,038.00  
ORÍGENES RETIRO                     0.00    13,988,283.00 25,290,424,558.00  
PROYECCIÓN RETIRO                   0.00     1,431,515.00    477,216,291.00  
SEGUNDA RETIRO                      0.00     8,967,561.00  2,623,315,789.00  
SMG RETIRO                          0.00    11,254,750.00  3,377,056,683.00  
ZURICH RETIRO                       0.00     1,595,847.00     25,373,604.00

In [59]:
Balance2.to_excel(r'C:\Users\eangenello\Desktop\Retiro_Varios.xlsx', sheet_name='Retiro')